In [ ]:
import sys
print(sys.version)
print(*sys.path, sep='\n')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision as tv

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

num_workers = os.cpu_count()
print(num_workers)

cpu
6


In [4]:
# parameters for trained CNN
root = os.path.join('.', 'dataset_root')
mean, std = [0.13066046], [0.30150425] # based on training set
batch_size = 64
lr = 0.01
epochs = 20

In [5]:
def imshow(tensor_grid, mean=0., std=1., title=None):
    assert isinstance(tensor_grid, torch.Tensor)
    assert len(tensor_grid.size()) == 4, \
        f'For a batch of images only, {tensor_grid.size()} '
    
    tensor_grid = tv.utils.make_grid(tensor_grid)
    grid = tensor_grid.numpy().transpose((1,2,0))
    grid = std * grid + mean
    grid = np.clip(grid, 0, 1)
    plt.imshow(grid)
    
    if title is not None:
        plt.title(title)
        
    plt.pause(0.001)

In [6]:
# prepare data
# foolbox model expects raw numpy array as image
transform = tv.transforms.Compose([
        tv.transforms.ToTensor(),
        tv.transforms.Normalize(mean, std)])
train_dataset = tv.datasets.MNIST(
    root,
    train=True, 
    download=True,
    transform=transform)
test_dataset = tv.datasets.MNIST(
    root,
    train=False, 
    download=True,
    transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(int(((28 - (3-1)) / 2)**2 * 8), 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [8]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0.
    corrects = 0
    
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        batch_size = x.size(0)
        
        optimizer.zero_grad()
        output = model(x)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()

        # for display
        total_loss += loss.item() * batch_size
        preds = output.max(1, keepdim=True)[1]
        corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(train_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [9]:
def validate(model, loader):
    model.eval()
    total_loss = 0.
    corrects = 0
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            batch_size = x.size(0)
            output = model(x)
            loss = F.nll_loss(output, y)
            total_loss += loss.item() * batch_size
            preds = output.max(1, keepdim=True)[1]
            corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(test_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [10]:
# NOTE: NO GPU AT SCHOOL!
model = Net()
model_softmax = torch.nn.Sequential(
    model,
    torch.nn.LogSoftmax(dim=1)
)
optimizer = torch.optim.SGD(model_softmax.parameters(), lr=lr)

In [11]:
images = next(iter(train_loader))[0]
model_softmax(images[:1])

tensor([[-2.1024, -2.3901, -2.4845, -2.1086, -2.1934, -2.4624, -2.2889, -2.4741,
         -2.2595, -2.3575]], grad_fn=<LogSoftmaxBackward>)

In [12]:
since = time.time()
for epoch in range(epochs):
    start = time.time()
    tr_loss, tr_acc = train(model_softmax, train_loader, optimizer)
    va_loss, va_acc = validate(model_softmax, test_loader)
    
    time_elapsed = time.time() - start
    print(('[{:2d}] {:.0f}m {:.1f}s Train Loss: {:.4f} Accuracy: {:.4f}%, ' +
        'Test Loss: {:.4f} Accuracy: {:.4f}%').format(
            epoch+1, time_elapsed // 60, time_elapsed % 60,
            tr_loss, tr_acc*100.,
            va_loss, va_acc*100.))
    
    # exit early
#     if va_acc > 0.98:
#         break
    
time_elapsed = time.time() - since
print('Total run time: {:.0f}m {:.1f}s'.format(
    time_elapsed // 60,
    time_elapsed % 60))

[ 1] 0m 11.5s Train Loss: 0.6072 Accuracy: 84.2850%, Test Loss: 0.2659 Accuracy: 92.3300%
[ 2] 0m 11.2s Train Loss: 0.2719 Accuracy: 92.5467%, Test Loss: 0.1838 Accuracy: 94.6000%
[ 3] 0m 11.5s Train Loss: 0.2041 Accuracy: 94.3833%, Test Loss: 0.1536 Accuracy: 95.3900%
[ 4] 0m 11.0s Train Loss: 0.1622 Accuracy: 95.5017%, Test Loss: 0.1399 Accuracy: 95.5500%
[ 5] 0m 11.1s Train Loss: 0.1390 Accuracy: 96.1033%, Test Loss: 0.1012 Accuracy: 96.9300%
[ 6] 0m 11.2s Train Loss: 0.1195 Accuracy: 96.6483%, Test Loss: 0.0941 Accuracy: 97.1700%
[ 7] 0m 11.1s Train Loss: 0.1075 Accuracy: 96.9850%, Test Loss: 0.0883 Accuracy: 97.2100%
[ 8] 0m 11.1s Train Loss: 0.0968 Accuracy: 97.3267%, Test Loss: 0.0901 Accuracy: 97.1500%
[ 9] 0m 11.1s Train Loss: 0.0882 Accuracy: 97.4433%, Test Loss: 0.0748 Accuracy: 97.6400%
[10] 0m 11.0s Train Loss: 0.0825 Accuracy: 97.6867%, Test Loss: 0.0671 Accuracy: 97.9900%
[11] 0m 11.0s Train Loss: 0.0754 Accuracy: 97.8983%, Test Loss: 0.0656 Accuracy: 97.7900%
[12] 0m 11

KeyboardInterrupt: 

In [ ]:
assert False
torch.save(model.state_dict(), 'mnist_simple_nosoftmax.pt')
torch.save(model_softmax.state_dict(), 'mnist_simple_full.pt')